<a href="https://colab.research.google.com/github/ljs7463/Dacon-HanRiverForecasting/blob/main/%EB%8D%B0%EC%9D%B4%EC%BD%98_LGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 모듈 Import

In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from glob import glob
from scipy import interpolate
import warnings
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose 
from sklearn.preprocessing import StandardScaler
from sklearn import neighbors
from sklearn.metrics import mean_squared_error 
from math import sqrt
from statsmodels.tsa.seasonal import seasonal_decompose 
warnings.filterwarnings("ignore")
sys.path.append(str(Path(os.getcwd())))

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Path 설정

In [2]:
base_path = Path(os.getcwd()).parent
sys.path.append(base_path)

# 폰트설정

In [3]:
plt.rc('font', family = 'Malgun Gothic')

# 데이터 로딩 및 병합

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# colab
rain_lst = glob('./drive/MyDrive/rf_data/*.csv')
water_lst = glob('./drive/MyDrive/water_data/*.csv')
water_df = pd.DataFrame()
rain_df = pd.DataFrame()
for w in water_lst:
    water_df = water_df.append(pd.read_csv(w))
for r in rain_lst:
    rain_df = rain_df.append(pd.read_csv(r))

In [6]:
# # jupyter
# water_lst = glob(f'{base_path}/datasource/competition_data/water_data/*.csv')
# rain_lst = glob(f'{base_path}/datasource/competition_data/rf_data/*.csv')

# water_df = pd.DataFrame()
# rain_df = pd.DataFrame()
# for w in water_lst:
#     water_df = water_df.append(pd.read_csv(w))
# for r in rain_lst:
#     rain_df = rain_df.append(pd.read_csv(r))

# 타입 수정 및 데이터 추가 병합

In [7]:
water_df.reset_index(drop = True,inplace = True)
rain_df.reset_index(drop = True,inplace = True)

In [8]:
# datetime
water_df['ymdhm'] = pd.to_datetime(water_df['ymdhm'], format = '%Y-%m-%d %H:%M:%S' )
rain_df['ymdhm'] = pd.to_datetime(water_df['ymdhm'], format = '%Y-%m-%d %H:%M:%S' )

# merge to dataframe
merge_df = pd.merge(water_df, rain_df, how = 'left', on = 'ymdhm')

# 컬럼 수정

In [9]:
new_cols ={'ymdhm':"date",
           'swl':'팔당댐 현재수위',
           'inf':'팔당댐 유입량',
           'sfw':'팔당댐 저수량',
           'ecpc':'팔당댐 공용량',
           'tototf':'총 방류량',
           'tide_level':"강화대교 조위",
           'fw_1018662':'청담대교 유량',
           'fw_1018680':'잠수교 유량',
           'fw_1018683':'한강대교 유량',
           'fw_1019630':'행주대교 유량',
           'wl_1018662':'청담대교 수위',    
           'wl_1018680':'잠수교 수위',
           'wl_1018683':"한강대교 수위",
           'wl_1019630':"행주대교 수위",
           'rf_10184100':'대곡교 강수량',
           'rf_10184110':'진관교 강수량',
           'rf_10184140':'송정동 강수량'}
merge_df =merge_df.rename(columns = new_cols)

# 데이터 분리 

In [10]:
merge_df

,date,팔당댐 현재수위,팔당댐 유입량,팔당댐 저수량,팔당댐 공용량,총 방류량,강화대교 조위,청담대교 수위,청담대교 유량,잠수교 수위,잠수교 유량,한강대교 수위,한강대교 유량,행주대교 수위,행주대교 유량,대곡교 강수량,진관교 강수량,송정동 강수량
0,2020-05-01 00:00:00,25.020,34.0,212.14,31.86,135.0,398.0,278.7,269.40,279.2,NaN,264.0,231.30,252.3,301.39,0.0,0.0,0.0
1,2020-05-01 00:10:00,25.010,0.0,211.77,32.23,135.0,386.0,278.7,269.40,279.2,NaN,264.0,219.12,252.3,301.39,0.0,0.0,0.0
2,2020-05-01 00:20:00,25.010,0.0,211.77,32.23,135.0,374.0,278.7,269.40,279.2,NaN,264.0,215.07,252.3,301.39,0.0,0.0,0.0
3,2020-05-01 00:30:00,25.010,0.0,211.77,32.23,135.0,363.0,278.7,269.40,279.2,NaN,264.0,219.12,251.3,292.61,0.0,0.0,0.0
4,2020-05-01 00:40:00,25.010,28.0,211.77,32.23,129.0,352.0,278.7,269.40,279.2,NaN,264.0,227.24,250.3,283.96,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276331,2013-10-31 23:10:00,25.231,171.9,234.40,9.60,171.9,109.0,280.7,280.22,274.2,0.0,268.0,372.96,259.3,366.60,0.0,0.0,0.0
276332,2013-10-31 23:20:00,25.224,71.0,234.10,9.90,171.0,124.0,280.7,280.22,274.2,0.0,268.0,348.63,259.3,366.60,0.0,0.0,0.0
276333,2013-10-31 23:30:00,25.219,70.6,233.91,10.09,170.6,141.0,280.7,280.22,273.2,0.0,267.0,315.76,260.3,376.45,0.0,0.0,0.0
276334,2013-10-31 23:40:00,25.219,70.2,233.91,10.09,170.2,157.0,280.7,280.22,273.2,0.0,267.0,263.13,260.3,376.45,0.0,0.0,0.0


In [11]:
# test set: 2022 06 01 ~
train = merge_df[-((merge_df['date'].dt.year==2022) & (merge_df['date'].dt.month>=6))].reset_index(drop =True)
test = merge_df[((merge_df['date'].dt.year==2022) & (merge_df['date'].dt.month>=6))].reset_index(drop =True)

In [12]:
train

,date,팔당댐 현재수위,팔당댐 유입량,팔당댐 저수량,팔당댐 공용량,총 방류량,강화대교 조위,청담대교 수위,청담대교 유량,잠수교 수위,잠수교 유량,한강대교 수위,한강대교 유량,행주대교 수위,행주대교 유량,대곡교 강수량,진관교 강수량,송정동 강수량
0,2020-05-01 00:00:00,25.020,34.0,212.14,31.86,135.0,398.0,278.7,269.40,279.2,NaN,264.0,231.30,252.3,301.39,0.0,0.0,0.0
1,2020-05-01 00:10:00,25.010,0.0,211.77,32.23,135.0,386.0,278.7,269.40,279.2,NaN,264.0,219.12,252.3,301.39,0.0,0.0,0.0
2,2020-05-01 00:20:00,25.010,0.0,211.77,32.23,135.0,374.0,278.7,269.40,279.2,NaN,264.0,215.07,252.3,301.39,0.0,0.0,0.0
3,2020-05-01 00:30:00,25.010,0.0,211.77,32.23,135.0,363.0,278.7,269.40,279.2,NaN,264.0,219.12,251.3,292.61,0.0,0.0,0.0
4,2020-05-01 00:40:00,25.010,28.0,211.77,32.23,129.0,352.0,278.7,269.40,279.2,NaN,264.0,227.24,250.3,283.96,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269419,2013-10-31 23:10:00,25.231,171.9,234.40,9.60,171.9,109.0,280.7,280.22,274.2,0.0,268.0,372.96,259.3,366.60,0.0,0.0,0.0
269420,2013-10-31 23:20:00,25.224,71.0,234.10,9.90,171.0,124.0,280.7,280.22,274.2,0.0,268.0,348.63,259.3,366.60,0.0,0.0,0.0
269421,2013-10-31 23:30:00,25.219,70.6,233.91,10.09,170.6,141.0,280.7,280.22,273.2,0.0,267.0,315.76,260.3,376.45,0.0,0.0,0.0
269422,2013-10-31 23:40:00,25.219,70.2,233.91,10.09,170.2,157.0,280.7,280.22,273.2,0.0,267.0,263.13,260.3,376.45,0.0,0.0,0.0


In [13]:
test

,date,팔당댐 현재수위,팔당댐 유입량,팔당댐 저수량,팔당댐 공용량,총 방류량,강화대교 조위,청담대교 수위,청담대교 유량,잠수교 수위,잠수교 유량,한강대교 수위,한강대교 유량,행주대교 수위,행주대교 유량,대곡교 강수량,진관교 강수량,송정동 강수량
0,2022-06-01 00:00:00,25.19,140.60,218.36,25.64,140.60,162.0,0.0,269.40,0.0,NaN,0.0,8.23,0.0,471.08,0.0,0.0,0.0
1,2022-06-01 00:10:00,25.19,140.78,218.36,25.64,140.78,151.0,0.0,280.22,0.0,NaN,0.0,28.82,0.0,449.12,0.0,0.0,0.0
2,2022-06-01 00:20:00,25.20,755.90,218.73,25.27,140.90,141.0,0.0,296.87,0.0,NaN,0.0,12.35,0.0,417.17,0.0,0.0,0.0
3,2022-06-01 00:30:00,25.19,0.00,218.36,25.64,140.94,130.0,0.0,302.53,0.0,NaN,0.0,53.52,0.0,386.43,0.0,0.0,0.0
4,2022-06-01 00:40:00,25.19,140.63,218.36,25.64,140.63,120.0,0.0,296.87,0.0,NaN,0.0,107.04,0.0,366.60,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6907,2022-07-18 23:10:00,25.04,259.23,212.86,31.14,259.23,510.0,0.0,319.84,0.0,NaN,0.0,-456.41,0.0,974.40,0.0,0.0,0.0
6908,2022-07-18 23:20:00,25.04,260.46,212.86,31.14,260.46,492.0,0.0,314.01,0.0,NaN,0.0,-717.30,0.0,1006.88,0.0,0.0,0.0
6909,2022-07-18 23:30:00,25.04,259.37,212.86,31.14,259.37,475.0,0.0,387.55,0.0,NaN,0.0,-843.37,0.0,1039.90,0.0,0.0,0.0
6910,2022-07-18 23:40:00,25.04,259.13,212.86,31.14,259.13,458.0,0.0,454.91,0.0,NaN,0.0,-1023.37,0.0,1073.46,0.0,0.0,0.0


# Interpolation

In [14]:
# 0이하의 값 일괄 nan값으로 변경(-값은 역류의 경우가 있다고 한다. 일단은 제거해보자)
for i in list(train.columns[1:6]):
    error_lst = list(train[train[i]<=0][i].index)
    train.loc[error_lst, i] = np.nan

In [15]:
train.isna().sum()

date             0
팔당댐 현재수위       790
팔당댐 유입량      19293
팔당댐 저수량        790
팔당댐 공용량        707
총 방류량         1159
강화대교 조위       4923
청담대교 수위         59
청담대교 유량      15164
잠수교 수위          59
잠수교 유량      189937
한강대교 수위         59
한강대교 유량       1279
행주대교 수위         59
행주대교 유량         59
대곡교 강수량          0
진관교 강수량          0
송정동 강수량          0
dtype: int64

## Method 1. interpolate

- date 컬럼을 인덱스 변환 후 진행 이후 reset_index() 로 인덱스 초기화

In [16]:
# train = train.set_index('date')
# train = train.interpolate(method = 'time')

In [17]:
# test = test.set_index('date')
# test = test.interpolate(method = 'time') # method : linear, time, values

In [18]:
# train =train.reset_index()
# test = test.reset_index()

## Method 2. Moving Average| + interpolate

In [19]:
train[(train['date'].dt.year==2013)].isna().sum()

date          0
팔당댐 현재수위     14
팔당댐 유입량     702
팔당댐 저수량      14
팔당댐 공용량       0
총 방류량        14
강화대교 조위      15
청담대교 수위       0
청담대교 유량       0
잠수교 수위        0
잠수교 유량        0
한강대교 수위       0
한강대교 유량       0
행주대교 수위       0
행주대교 유량       0
대곡교 강수량       0
진관교 강수량       0
송정동 강수량       0
dtype: int64

In [20]:
# 2012년 
df_2012 = train[(train['date'].dt.year==2012)]
df_2012 = df_2012.fillna(df_2012.rolling(3, min_periods = 1, axis = 0,center = True).mean())
df_2012 = df_2012.set_index('date').interpolate(method = 'time').reset_index()# 강화대교 조위만 해당
# 2013년 
df_2013 = train[(train['date'].dt.year==2013)].reset_index(drop = True)
df_2013 = df_2013.fillna(df_2013.rolling(3, min_periods = 1, axis = 0,center = True).mean())
df_2013 = df_2013.fillna(df_2013.rolling(6, min_periods = 1, axis = 0,center = True).mean())
# 2014년
df_2014 = train[(train['date'].dt.year==2014)].reset_index(drop = True)
df_2014 = df_2014.fillna(df_2014.rolling(3, min_periods = 1, axis = 0,center = True).mean())
df_2014 = df_2014.fillna(df_2014.rolling(6, min_periods = 1, axis = 0,center = True).mean())
# 2015년
df_2015 = train[(train['date'].dt.year==2015)].reset_index(drop = True)
df_2015 = df_2015.fillna(df_2015.rolling(3, min_periods = 1, axis = 0,center = True).mean())
df_2015 = df_2015.fillna(df_2015.rolling(6, min_periods = 1, axis = 0,center = True).mean())
df_2015 = df_2015.fillna(df_2015.rolling(8, min_periods = 1, axis = 0,center = True).mean())
df_2015 = df_2015.fillna(df_2015.rolling(10, min_periods = 1, axis = 0,center = True).mean())
df_2015 = df_2015.set_index('date').interpolate(method = 'time').reset_index()
# 2016년
df_2016 = train[(train['date'].dt.year==2016)].reset_index(drop = True)
df_2016 = df_2016.fillna(df_2016.rolling(3, min_periods = 1, axis = 0,center = True).mean())
df_2016 = df_2016.fillna(df_2016.rolling(6, min_periods = 1, axis = 0,center = True).mean())
df_2016 = df_2016.fillna(df_2016.rolling(8, min_periods = 1, axis = 0,center = True).mean())
df_2016 = df_2016.set_index('date').interpolate(method = 'time').reset_index()
# 2017년
df_2017 = train[(train['date'].dt.year==2017)].reset_index(drop = True)
df_2017 = df_2017.fillna(df_2017.rolling(3, min_periods = 1, axis = 0,center = True).mean())
df_2017 = df_2017.fillna(df_2017.rolling(6, min_periods = 1, axis = 0,center = True).mean())
df_2017 = df_2017.fillna(df_2017.rolling(8, min_periods = 1, axis = 0,center = True).mean())
df_2017 = df_2017.set_index('date').interpolate(method = 'time').reset_index()
# 2018년
df_2018 = train[(train['date'].dt.year==2018)].reset_index(drop = True)
df_2018 = df_2018.fillna(df_2018.rolling(3, min_periods = 1, axis = 0,center = True).mean())
df_2018 = df_2018.fillna(df_2018.rolling(6, min_periods = 1, axis = 0,center = True).mean())
df_2018 = df_2018.fillna(df_2018.rolling(8, min_periods = 1, axis = 0,center = True).mean())
df_2018 = df_2018.set_index('date').interpolate(method = 'time').reset_index()
# 2019년
df_2019 = train[(train['date'].dt.year==2019)].reset_index(drop = True)
df_2019 = df_2019.fillna(df_2019.rolling(3, min_periods = 1, axis = 0,center = True).mean())
df_2019 = df_2019.fillna(df_2019.rolling(6, min_periods = 1, axis = 0,center = True).mean())
df_2019 = df_2019.fillna(df_2019.rolling(8, min_periods = 1, axis = 0,center = True).mean())
df_2019 = df_2019.set_index('date').interpolate(method = 'time').reset_index()
# 2020년
df_2020 = train[(train['date'].dt.year==2020)].reset_index(drop = True)
df_2020 = df_2020.fillna(df_2020.rolling(3, min_periods = 1, axis = 0,center = True).mean())
df_2020 = df_2020.fillna(df_2020.rolling(6, min_periods = 1, axis = 0,center = True).mean())
df_2020 = df_2020.fillna(df_2020.rolling(8, min_periods = 1, axis = 0,center = True).mean())
df_2020 = df_2020.set_index('date').interpolate(method = 'time').reset_index()
# 2021년
df_2021 = train[(train['date'].dt.year==2021)].reset_index(drop = True)
df_2021 = df_2021.fillna(df_2021.rolling(3, min_periods = 1, axis = 0,center = True).mean())
df_2021 = df_2021.fillna(df_2021.rolling(6, min_periods = 1, axis = 0,center = True).mean())
df_2021 = df_2021.fillna(df_2021.rolling(8, min_periods = 1, axis = 0,center = True).mean())
df_2021 = df_2021.set_index('date').interpolate(method = 'time').reset_index()
# 2022년
df_2022 = train[(train['date'].dt.year==2022)].reset_index(drop = True)
df_2022 = df_2022.fillna(df_2022.rolling(3, min_periods = 1, axis = 0,center = True).mean())
df_2022 = df_2022.fillna(df_2022.rolling(6, min_periods = 1, axis = 0,center = True).mean())
df_2022 = df_2022.fillna(df_2022.rolling(8, min_periods = 1, axis = 0,center = True).mean())
df_2022 = df_2022.set_index('date').interpolate(method = 'time').reset_index()

In [21]:
# train_df = pd.concat([df_2012,df_2013],axis = 0).reset_index(drop = True)
# train_df = pd.concat([train_df,df_2014],axis = 0).reset_index(drop = True)
# train_df = pd.concat([train_df,df_2015],axis = 0).reset_index(drop = True)
# train_df = pd.concat([train_df,df_2016],axis = 0).reset_index(drop = True)
# train_df = pd.concat([train_df,df_2017],axis = 0).reset_index(drop = True)
# train_df = pd.concat([train_df,df_2018],axis = 0).reset_index(drop = True)
# train_df = pd.concat([train_df,df_2019],axis = 0).reset_index(drop = True)
# train_df = pd.concat([train_df,df_2020],axis = 0).reset_index(drop = True)
# train_df = pd.concat([train_df,df_2021],axis = 0).reset_index(drop = True)
# train_df = pd.concat([train_df,df_2022],axis = 0).reset_index(drop = True)

In [22]:
train_df = df_2013.drop(columns=['잠수교 유량', '강화대교 조위'])

In [23]:
# train_df =train_df.drop(columns = ['잠수교 유량','강화대교 조위'])

**데이터 분리**

In [24]:
# train_df = train_df.iloc[:2000]

In [25]:
from sklearn.model_selection import train_test_split
x_data = train_df[['팔당댐 현재수위', '팔당댐 유입량', '팔당댐 저수량', '팔당댐 공용량', '총 방류량', '청담대교 유량','한강대교 유량', '행주대교 유량', '대곡교 강수량','진관교 강수량','송정동 강수량']]
y_data =train_df[['청담대교 수위','잠수교 수위', '한강대교 수위', '행주대교 수위']]
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3) 

In [26]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((18547, 11), (7949, 11), (18547, 4), (7949, 4))

In [27]:
x_train = x_train.reset_index(drop = True)
x_test = x_test.reset_index(drop = True)
y_train = y_train.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)

In [28]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaler = pd.DataFrame(scaler.fit_transform(x_train), columns = x_train.columns)
x_test_scaler = pd.DataFrame(scaler.transform(x_test), columns = x_test.columns)

In [29]:
x_train_scaler.shape, y_train.shape

((18547, 11), (18547, 4))

In [30]:
y_train1 = y_train['청담대교 수위']
y_train2 = y_train['잠수교 수위']
y_train3 = y_train['한강대교 수위']
y_train4 = y_train['행주대교 ]수위']
y_test1 = y_test['청담대교 수위'
y_test2 = y_test['잠수교 수위']
y_test3 = y_test['한강대교 수위']
y_test4 = y_test['행주대교 수위']

In [31]:
y_train1=y_train1.astype(int)
y_train2=y_train2.astype(int)
y_train3=y_train3.astype(int)
y_train4=y_train4.astype(int)
# 왜 라벨은 float이 안되지???

In [32]:
# y_train = y_train.astype(int)

### Light Gradient Boost Model (non gridsearch)

In [ ]:
import lightgbm as lgb

lgb_model1 = lgb.LGBMRegressor(
    categorical_feature= [0,2,3,4,8,9,10],
    task = 'predict',
    application = 'regression',
    objective = 'root_mean_squared_error',
    boosting_type="gbdt",
    num_iterations = 2500,
    learning_rate = 0.05,
    num_leaves=15,
    tree_learner='feature',
    max_depth =10,
    min_data_in_leaf=7,
    bagging_fraction = 1,
    bagging_freq = 100,
    reg_sqrt='True',
    metric ='rmse',
    feature_fraction = 0.6,
    random_state=42)

lgb_model2 = lgb.LGBMRegressor(
    categorical_feature= [0,2,3,4,8,9,10],
    task = 'predict',
    application = 'regression',
    objective = 'root_mean_squared_error',
    boosting_type="gbdt",
    num_iterations = 2500,
    learning_rate = 0.05,
    num_leaves=15,
    tree_learner='feature',
    max_depth =10,
    min_data_in_leaf=7,
    bagging_fraction = 1,
    bagging_freq = 100,
    reg_sqrt='True',
    metric ='rmse',
    feature_fraction = 0.6,
    random_state=42)

lgb_model3 = lgb.LGBMRegressor(
    categorical_feature= [0,2,3,4,8,9,10],
    task = 'predict',
    application = 'regression',
    objective = 'root_mean_squared_error',
    boosting_type="gbdt",
    num_iterations = 2500,
    learning_rate = 0.05,
    num_leaves=15,
    tree_learner='feature',
    max_depth =10,
    min_data_in_leaf=7,
    bagging_fraction = 1,
    bagging_freq = 100,
    reg_sqrt='True',
    metric ='rmse',
    feature_fraction = 0.6,
    random_state=42)

lgb_model4 = lgb.LGBMRegressor(
    categorical_feature= [0,2,3,4,8,9,10],
    task = 'predict',
    application = 'regression',
    objective = 'root_mean_squared_error',
    boosting_type="gbdt",
    num_iterations = 2500,
    learning_rate = 0.05,
    num_leaves=15,
    tree_learner='feature',
    max_depth =10,
    min_data_in_leaf=7,
    bagging_fraction = 1,
    bagging_freq = 100,
    reg_sqrt='True',
    metric ='rmse',
    feature_fraction = 0.6,
    random_state=42)

lgb_model1.fit(x_train_scaler,y_train1) 
preds_lgb_model1 = lgb_model1.predict(x_test_scaler)
rmse_lgb1 = np.sqrt(mean_squared_error(y_test1, preds_lgb_model1))
print(" RMSE1: %f" % (rmse_lgb1 ))

lgb_model2.fit(x_train_scaler,y_train2) 
preds_lgb_model2 = lgb_model2.predict(x_test_scaler)
rmse_lgb2 = np.sqrt(mean_squared_error(y_test2, preds_lgb_model2))
print(" RMSE2: %f" % (rmse_lgb2 ))

lgb_model3.fit(x_train_scaler,y_train3) 
preds_lgb_model3 = lgb_model3.predict(x_test_scaler)
rmse_lgb3 = np.sqrt(mean_squared_error(y_test3, preds_lgb_model3))
print(" RMSE3: %f" % (rmse_lgb3 ))

lgb_model4.fit(x_train_scaler,y_train4) 
preds_lgb_model4 = lgb_model4.predict(x_test_scaler)
rmse_lgb4 = np.sqrt(mean_squared_error(y_test4, preds_lgb_model4))
print(" RMSE4: %f" % (rmse_lgb4 ))

 RMSE1: 1.731830
 RMSE2: 2.555071
 RMSE3: 2.524025
 RMSE4: 0.749680


In [ ]:
test =test.drop(columns = ['잠수교 유량','강화대교 조위'])
test.isna().sum()

date           0
팔당댐 현재수위      36
팔당댐 유입량       36
팔당댐 저수량       36
팔당댐 공용량       36
총 방류량         36
청담대교 수위        0
청담대교 유량     1216
잠수교 수위         0
한강대교 수위        0
한강대교 유량        0
행주대교 수위        0
행주대교 유량        0
대곡교 강수량        0
진관교 강수량        0
송정동 강수량        0
dtype: int64

In [ ]:
x_data = test[['date','팔당댐 현재수위', '팔당댐 유입량', '팔당댐 저수량', '팔당댐 공용량', '총 방류량', '청담대교 유량', '한강대교 유량','행주대교 유량', '대곡교 강수량', '진관교 강수량', '송정동 강수량']]
y_data = test[['청담대교 수위', '잠수교 수위', '한강대교 수위', '행주대교 수위']]

In [ ]:
# 0이하의 값 일괄 nan값으로 변경
for i in list(x_data.columns[1:6]):
    error_lst = list(x_data[x_data[i]<=0][i].index)
    x_data.loc[error_lst, i] = np.nan

In [ ]:
x_data.isna().sum()

date           0
팔당댐 현재수위      36
팔당댐 유입량      566
팔당댐 저수량       36
팔당댐 공용량       36
총 방류량         36
청담대교 유량     1216
한강대교 유량        0
행주대교 유량        0
대곡교 강수량        0
진관교 강수량        0
송정동 강수량        0
dtype: int64

In [ ]:
x_data = x_data.fillna(x_data.rolling(3, min_periods = 1, axis = 0, center = True).mean())
x_data = x_data.fillna(x_data.rolling(6, min_periods = 1, axis = 0, center = True).mean())
x_data = x_data.fillna(x_data.rolling(8, min_periods = 1, axis = 0, center = True).mean())
x_data = x_data.fillna(x_data.rolling(10, min_periods = 1, axis = 0, center = True).mean())
x_data = x_data.set_index('date').interpolate(method = 'time').reset_index()
x_data =x_data.drop(columns = 'date') 

In [ ]:
x_data

,팔당댐 현재수위,팔당댐 유입량,팔당댐 저수량,팔당댐 공용량,총 방류량,청담대교 유량,한강대교 유량,행주대교 유량,대곡교 강수량,진관교 강수량,송정동 강수량
0,25.19,140.600,218.36,25.64,140.60,269.40,8.23,471.08,0.0,0.0,0.0
1,25.19,140.780,218.36,25.64,140.78,280.22,28.82,449.12,0.0,0.0,0.0
2,25.20,755.900,218.73,25.27,140.90,296.87,12.35,417.17,0.0,0.0,0.0
3,25.19,448.265,218.36,25.64,140.94,302.53,53.52,386.43,0.0,0.0,0.0
4,25.19,140.630,218.36,25.64,140.63,296.87,107.04,366.60,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
6907,25.04,259.230,212.86,31.14,259.23,319.84,-456.41,974.40,0.0,0.0,0.0
6908,25.04,260.460,212.86,31.14,260.46,314.01,-717.30,1006.88,0.0,0.0,0.0
6909,25.04,259.370,212.86,31.14,259.37,387.55,-843.37,1039.90,0.0,0.0,0.0
6910,25.04,259.130,212.86,31.14,259.13,454.91,-1023.37,1073.46,0.0,0.0,0.0


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_data_scaler = pd.DataFrame(scaler.fit_transform(x_data), columns = x_data.columns)

In [ ]:
x_data_scaler

,팔당댐 현재수위,팔당댐 유입량,팔당댐 저수량,팔당댐 공용량,총 방류량,청담대교 유량,한강대교 유량,행주대교 유량,대곡교 강수량,진관교 강수량,송정동 강수량
0,1.574942,-0.509080,1.564554,-1.564554,-0.480121,-0.421466,-0.497826,-0.434363,-0.156112,-0.152065,-0.15499
1,1.574942,-0.508940,1.564554,-1.564554,-0.479970,-0.413926,-0.485315,-0.447857,-0.156112,-0.152065,-0.15499
2,1.654771,-0.030754,1.644765,-1.644765,-0.479870,-0.402323,-0.495323,-0.467489,-0.156112,-0.152065,-0.15499
3,1.574942,-0.269905,1.564554,-1.564554,-0.479837,-0.398379,-0.470307,-0.486377,-0.156112,-0.152065,-0.15499
4,1.574942,-0.509057,1.564554,-1.564554,-0.480096,-0.402323,-0.437786,-0.498562,-0.156112,-0.152065,-0.15499
...,...,...,...,...,...,...,...,...,...,...,...
6907,0.377507,-0.416859,0.372235,-0.372235,-0.380953,-0.386316,-0.780154,-0.125096,-0.156112,-0.152065,-0.15499
6908,0.377507,-0.415903,0.372235,-0.372235,-0.379925,-0.390379,-0.938678,-0.105138,-0.156112,-0.152065,-0.15499
6909,0.377507,-0.416750,0.372235,-0.372235,-0.380836,-0.339131,-1.015282,-0.084849,-0.156112,-0.152065,-0.15499
6910,0.377507,-0.416936,0.372235,-0.372235,-0.381037,-0.292190,-1.124655,-0.064228,-0.156112,-0.152065,-0.15499


In [ ]:
y_pred1 = lgb_model1.predict(x_data_scaler)
y_pred2 = lgb_model2.predict(x_data_scaler)
y_pred3 = lgb_model3.predict(x_data_scaler)
y_pred4 = lgb_model4.predict(x_data_scaler)
y_pred1 = pd.DataFrame(y_pred1)
y_pred2 = pd.DataFrame(y_pred2)
y_pred3 = pd.DataFrame(y_pred3)
y_pred4 = pd.DataFrame(y_pred4)
y_pred1 = y_pred1.rename(columns = {0:'wl_1018662'})
y_pred2 = y_pred2.rename(columns = {0:'wl_1018680'})
y_pred3 = y_pred3.rename(columns = {0:'wl_1018683'})
y_pred4 = y_pred4.rename(columns = {0:'wl_1019630'})

In [ ]:
index_date = test['date']

In [ ]:
index_date

0      2022-06-01 00:00:00
1      2022-06-01 00:10:00
2      2022-06-01 00:20:00
3      2022-06-01 00:30:00
4      2022-06-01 00:40:00
               ...        
6907   2022-07-18 23:10:00
6908   2022-07-18 23:20:00
6909   2022-07-18 23:30:00
6910   2022-07-18 23:40:00
6911   2022-07-18 23:50:00
Name: date, Length: 6912, dtype: datetime64[ns]

In [ ]:
result =pd.concat([index_date, y_pred1],axis = 1)
result =pd.concat([result, y_pred2],axis = 1)
result =pd.concat([result, y_pred3],axis = 1)
result =pd.concat([result, y_pred4],axis = 1)
result

,date,wl_1018662,wl_1018680,wl_1018683,wl_1019630
0,2022-06-01 00:00:00,300.793382,296.033738,289.516926,282.064537
1,2022-06-01 00:10:00,303.046390,296.244739,289.353022,280.146313
2,2022-06-01 00:20:00,305.313628,296.556926,289.593240,277.259792
3,2022-06-01 00:30:00,305.297754,295.854074,289.471899,275.142643
4,2022-06-01 00:40:00,305.260787,295.811490,287.717007,272.879530
...,...,...,...,...,...
6907,2022-07-18 23:10:00,308.869924,312.629685,310.764992,314.941198
6908,2022-07-18 23:20:00,307.583482,312.692441,310.985218,316.849075
6909,2022-07-18 23:30:00,316.049012,317.953378,316.818486,320.733779
6910,2022-07-18 23:40:00,326.609144,323.628572,323.483944,321.075918


In [ ]:
result  = result.rename(columns = {'date':'ymdhm'})
result

,ymdhm,wl_1018662,wl_1018680,wl_1018683,wl_1019630
0,2022-06-01 00:00:00,300.793382,296.033738,289.516926,282.064537
1,2022-06-01 00:10:00,303.046390,296.244739,289.353022,280.146313
2,2022-06-01 00:20:00,305.313628,296.556926,289.593240,277.259792
3,2022-06-01 00:30:00,305.297754,295.854074,289.471899,275.142643
4,2022-06-01 00:40:00,305.260787,295.811490,287.717007,272.879530
...,...,...,...,...,...
6907,2022-07-18 23:10:00,308.869924,312.629685,310.764992,314.941198
6908,2022-07-18 23:20:00,307.583482,312.692441,310.985218,316.849075
6909,2022-07-18 23:30:00,316.049012,317.953378,316.818486,320.733779
6910,2022-07-18 23:40:00,326.609144,323.628572,323.483944,321.075918


In [ ]:
import datetime
result['ymdhm'] = result['ymdhm'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M'))

In [ ]:
result

,ymdhm,wl_1018662,wl_1018680,wl_1018683,wl_1019630
0,2022-06-01 00:00,300.793382,296.033738,289.516926,282.064537
1,2022-06-01 00:10,303.046390,296.244739,289.353022,280.146313
2,2022-06-01 00:20,305.313628,296.556926,289.593240,277.259792
3,2022-06-01 00:30,305.297754,295.854074,289.471899,275.142643
4,2022-06-01 00:40,305.260787,295.811490,287.717007,272.879530
...,...,...,...,...,...
6907,2022-07-18 23:10,308.869924,312.629685,310.764992,314.941198
6908,2022-07-18 23:20,307.583482,312.692441,310.985218,316.849075
6909,2022-07-18 23:30,316.049012,317.953378,316.818486,320.733779
6910,2022-07-18 23:40,326.609144,323.628572,323.483944,321.075918


In [ ]:
result = result.set_index('ymdhm')

In [ ]:
result.to_csv('제발되라진짜.csv', encoding = 'UTF-8')

In [ ]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6912 entries, 0 to 6911
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ymdhm       6912 non-null   datetime64[ns]
 1   wl_1018662  6912 non-null   float64       
 2   wl_1018680  6912 non-null   float64       
 3   wl_1018683  6912 non-null   float64       
 4   wl_1019630  6912 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 270.1 KB


In [ ]:
result.to_csv('submission_js_.csv')

### Light Gradient Boost Model(grid search)

In [45]:
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
parameters = {
    'task' : ['predict'],
    'boosting': ['gbdt' ],
    'objective': ['root_mean_squared_error'],
    'num_iterations': [  1500, 2000,5000  ],
    'learning_rate':[  0.01, 0.05, 0.1 ],
   'num_leaves':[ 7, 15, 31  ],
   'max_depth' :[ 10,15,25],
   'min_data_in_leaf':[15,25 ],
  'feature_fraction': [ 0.6, 0.8,  0.9],
    'bagging_fraction': [  0.6, 0.8 ],
    'bagging_freq': [ 50,  100, 200],
}

grid = GridSearchCV(lgb.LGBMRegressor(random_state=0), parameters, scoring='neg_mean_absolute_error', cv=5)

# fit and predict and Performance each model
lgb_model1 = grid.fit(x_train_scaler, y_train1)
pred_lgb_model1 = lgb_model1.predict(x_test_scaler)
rmse_lgb1 = np.sqrt(mean_squared_error(y_test1, preds_lgb_model1))
print(f'RMSE1: {rmse_lgb1}')

lgb_model2 = grid.fit(x_train_scaler, y_train2)
pred_lgb_model2 = lgb_model2.predict(x_test_scaler)
rmse_lgb2 = np.sqrt(mean_squared_error(y_test2, preds_lgb_model2))
print(f'RMSE2: {rmse_lgb2}')

lgb_model3 = grid.fit(x_train_scaler, y_train3)
pred_lgb_model3 = lgb_model3.predict(x_test_scaler)
rmse_lgb3 = np.sqrt(mean_squared_error(y_test3, preds_lgb_model3))
print(f'RMSE3: {rmse_lgb3}')

lgb_model4 = grid.fit(x_train_scaler, y_train4)
pred_lgb_model4 = lgb_model4.predict(x_test_scaler)
rmse_lgb4 = np.sqrt(mean_squared_error(y_test4, preds_lgb_model4))
print(f'RMSE4: {rmse_lgb4}')


KeyboardInterrupt: ignored

In [34]:
import lightgbm as lgb
lgb_model = lgb.LGBMRegressor(
    task = 'predict',
    application = 'regression',
    objective = 'root_mean_squared_error',
    boosting_type="gbdt",
    num_iterations = 1000,
    learning_rate = 0.01,
    num_leaves=15,
    max_depth =10,
    min_data_in_leaf=15,
    bagging_fraction = 0.6,
    reg_sqrt='True',
    metric ='rmse',
    feature_fraction = 1.0,
    random_state=0)


In [35]:
test =test.drop(columns = ['잠수교 유량','강화대교 조위'])
test.isna().sum()

date           0
팔당댐 현재수위      36
팔당댐 유입량       36
팔당댐 저수량       36
팔당댐 공용량       36
총 방류량         36
청담대교 수위        0
청담대교 유량     1216
잠수교 수위         0
한강대교 수위        0
한강대교 유량        0
행주대교 수위        0
행주대교 유량        0
대곡교 강수량        0
진관교 강수량        0
송정동 강수량        0
dtype: int64

In [36]:
x_data = test[['date','팔당댐 현재수위', '팔당댐 유입량', '팔당댐 저수량', '팔당댐 공용량', '총 방류량', '청담대교 유량', '한강대교 유량','행주대교 유량', '대곡교 강수량', '진관교 강수량', '송정동 강수량']]
y_data = test[['청담대교 수위', '잠수교 수위', '한강대교 수위', '행주대교 수위']]

In [37]:
# 0이하의 값 일괄 nan값으로 변경
for i in list(x_data.columns[1:6]):
    error_lst = list(x_data[x_data[i]<=0][i].index)
    x_data.loc[error_lst, i] = np.nan

In [38]:
x_data.isna().sum()

date           0
팔당댐 현재수위      36
팔당댐 유입량      566
팔당댐 저수량       36
팔당댐 공용량       36
총 방류량         36
청담대교 유량     1216
한강대교 유량        0
행주대교 유량        0
대곡교 강수량        0
진관교 강수량        0
송정동 강수량        0
dtype: int64

In [39]:
x_data = x_data.fillna(x_data.rolling(3, min_periods = 1, axis = 0, center = True).mean())
x_data = x_data.fillna(x_data.rolling(6, min_periods = 1, axis = 0, center = True).mean())
x_data = x_data.fillna(x_data.rolling(8, min_periods = 1, axis = 0, center = True).mean())
x_data = x_data.fillna(x_data.rolling(10, min_periods = 1, axis = 0, center = True).mean())
x_data = x_data.set_index('date').interpolate(method = 'time').reset_index()
x_data =x_data.drop(columns = 'date') 

In [43]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_data_scaler = pd.DataFrame(scaler.fit_transform(x_data), columns = x_data.columns)

In [ ]:
y_pred1 = lgb_model.predict(x_data_scaler)
y_pred1 = pd.DataFrame(y_pred1)
y_pred1 = y_pred1.rename(columns = {0:'wl_1018662'})


y_pred2 = lgb_model.predict(x_data_scaler)
y_pred2 = pd.DataFrame(y_pred2)

y_pred3 = lgb_model.predict(x_data_scaler)

y_pred4 = lgb_model.predict(x_data_scaler)


y_pred2 = pd.DataFrame(y_pred2)
y_pred3 = pd.DataFrame(y_pred3)
y_pred4 = pd.DataFrame(y_pred4)
y_pred1 = y_pred1.rename(columns = {0:'wl_1018662'})
y_pred2 = y_pred2.rename(columns = {0:'wl_1018680'})
y_pred3 = y_pred3.rename(columns = {0:'wl_1018683'})
y_pred4 = y_pred4.rename(columns = {0:'wl_1019630'})

In [ ]:
index_date = test['date']

In [ ]:
result =pd.concat([index_date, y_pred1],axis = 1)
result =pd.concat([result, y_pred2],axis = 1)
result =pd.concat([result, y_pred3],axis = 1)
result =pd.concat([result, y_pred4],axis = 1)
result

In [ ]:
result  = result.rename(columns = {'date':'ymdhm'})
result

In [ ]:
import datetime
result['ymdhm'] = result['ymdhm'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M'))